# Exercise 2

The goal of the exercise is to implement the MPPI algorithm discussed in class to control a (simplified) 3 degrees of freedom manipulator.

You do not have access to the dynamics of the manipulator but only to a simulator (the ``simulate`` function) which you can use to test how the manipulator would behave for a given control sequence.
* The states $x$ of the manipulator are the angles (in radians) of the 3 joints.
* The controls $u$ set the velocity of the joints (3 controls).
* The tip of the manipulator is $p(x)$ and depends on the current states $x$ (this is computed using a ``forward_kinematics`` function).
* We assume a single shooting problem where we only optimize the control sequence $\bar{u} = u_0,u_1,\cdots$ and compute the resulting states using the simulator

The goal is to get the manipulator's tip $p(x)$ to reach a desired goal position by minimizing (with respect to $\bar{u}$)
$$l(\bar{u}) = \sum_{n=0}^{N_{horizon}} (p(x_n) - p_{desired})^2 + 0.01 u_n^2$$

To do so, you will implement a model-predictive controller using the MPPI algorithm in its inner loop.

## MPPI algorithm
At each iteration, the MPPI algorithm computes an improved control through
$$\bar{u} \leftarrow \bar{u} - \alpha \frac{ - \lambda \mathbb E \left[  \exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon) \right) \Sigma^{-1}  \epsilon \right]}{ \mu \mathbb   E \left[ \exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon) \right) \right]  }$$

which is approximated through Monte-Carlo sampling by
$$\bar{u} \leftarrow \bar{u} - \alpha \frac{ - \lambda \mathbb \sum_{k=1}^K \left(\exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon_k) \right) \Sigma^{-1}  \epsilon_k \right)}{ \mu \mathbb   \sum_{k=1}^K \left( \exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon_k) \right) \right)  }$$


Tips to implement the algorithm:
* The parameters $\alpha = 1$, $\Sigma = I$, $\lambda=0.1$, $\mu=2$ and $K=50$ work well for this problem (use the same samples to evaluate both sums)
* Do only one update of $\bar{u}$ per control cycle (i.e. one descent step) and reuse the previous trajectory as an initial guess shifted by one time-step (for the warm start)
* Use $N_{horizon} = 10$ (optimize 10 time steps ahead)
* Use the numpy function ``random.multivariate_normal`` to draw a random sample from a multi-dimensional Gaussian distribution (you will need to draw $3N_{horizon}$ samples for $\epsilon$)

## Questions
1. Implement the MPPI algorithm and use it to reach the following positions $(2,1)$, $(2,2)$, $(1,-2)$ (one full run per position). Run the simulation for 500 time steps.
2. How does the algorithm perform across different runs? Do you always get the same result? Why?
3. Plot the average cost and its variance across 10 runs for each target position

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import manipulator

In [ ]:
## the manipulator package contains all the necessary functions to simulate the robot

## the forward_kinematics function returns the position of the tip of the manipulator as a function of the joint positions
joint_position = np.array([0.,0.,0.])
tip_position = manipulator.forward_kinematics(joint_position)
print(f'the tip of the manipulator is at x={tip_position[0]:0.1f}, y={tip_position[1]:0.1f} ' + \
      f'when the joints are at {joint_position[0]:0.1f}, {joint_position[2]:0.1f}, {joint_position[2]:0.1f}')

joint_position = np.array([0.5,0.5,0.5])
tip_position = manipulator.forward_kinematics(joint_position)
print(f'the tip of the manipulator is at x={tip_position[0]:0.1f}, y={tip_position[1]:0.1f} ' + \
      f'when the joints are at {joint_position[0]:0.1f}, {joint_position[2]:0.1f}, {joint_position[2]:0.1f}')

In [ ]:
## the manipulator can be simulated with a control trajectory of length N

## the simulate function returns the time, state trajectory [3,N] and control trajectory [3,N]

N = 500 #number of timesteps
initial_state = np.array([0.,0.,0.])

## we compute a random control trajectory drawn from a multivariate normal distribution
u = np.reshape(np.random.multivariate_normal(np.zeros(3*N), np.eye(3*N)),[3,N])
t,x, u = manipulator.simulate(initial_state, u, N)

## the states can be displayed using the animate_robot function as follows
manipulator.animate_robot(x)

In [ ]:
## fill in the controller function
def controller(states):
    ''' simplistic MPC loop '''
    Nhorizon = 10
    u_guess = np.reshape(np.random.multivariate_normal(np.zeros(3*Nhorizon), np.eye(3*Nhorizon)),[3,Nhorizon])
    
    return u_guess[:,0]


## run the MPC controller
N_timesteps = 500
t, states, control = manipulator.simulate(np.zeros([3,]), controller, N_timesteps)

## display the results
manipulator.animate_robot(states)

## Note that the simulate function can take either a control trajectory of size [3,N] 
## or a function f(states) that will be called at each control cycle (cf. examples above)